In [ ]:
remotes::install_github("jinworks/CellChat@534b91d")

suppressPackageStartupMessages({
    library(CellChat)
    library(zellkonverter)
    library(SingleCellExperiment())
})

cellchat <- readRDS("tmp/cellchat.rds")
sce = readH5AD("tmp/adata_gated.h5ad")

In [ ]:
# change the order of the labels
cellchat@meta$labels <- forcats::fct_relevel(cellchat@meta$labels, "P14 top", "P14 crypt", "P14 muscularis")
cellchat <- updateClusterLabels(cellchat, new.order = levels(cellchat@meta$labels))

In [ ]:
# Define colors
color.use = colors = c(
    "Epithelial_Secretory" = "#AA9228",
    "Epithelial_Absorptive" = "#E3C300",
    "Monocyte" = "#C37698",
    "T-Cell" = "#008E74",
    "MAIT" = "#63ABB9",
    "Myeloid" = "#EF9684",
    "ILC" = "#A0C6D3",
    "B-Cell" = "#E2CEAB",
    "DC" = "#FE757D",
    "Fibroblast" = "#E17300",
    "Endothelial" = "#E30133",
    "NK" = "#4A7B89",
    "Epithelial_Progenitor" = "#F7BC00",
    "Neuron" = "#2A2446",
    "Erythroid" = "#A5021D",
    "Eosinophil" = "#782c4e"
)

subtype_to_type <- colData(sce)[c('Subtype_gate', 'Type')] |>
    as.data.frame() |>
    dplyr::distinct()

color.use <- color.use |> as.data.frame() |> tibble::rownames_to_column("Type")

subtype_to_type <- subtype_to_type |> left_join(color.use)

idx <- match(levels(cellchat@meta$labels), subtype_to_type$Subtype_gate)

In [ ]:
color.use <- subtype_to_type$color.use[idx]

## Figure 3g and extended data figure 3d

In [ ]:
netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming")

In [ ]:
netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing")

## Extended data figure 4a

In [ ]:
pathways.show <- c("TGFb") 

vertex.receiver	<- which(levels(cellchat@idents) %in% c("P14 top", "P14 crypt", "P14 muscularis"))

netVisual_individual(cellchat, signaling = pathways.show, layout = 'circle', targets.use=vertex.receiver, color.use=color.use)
